<a href="https://colab.research.google.com/github/muajnstu/DSK-Chain-to-predict-diabeties-/blob/main/Freidman_Test_on_refined_method_ipyn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-posthocs

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import friedmanchisquare
from scipy.stats import rankdata
import scikit_posthocs as sp
import altair as alt
from scipy.stats import friedmanchisquare, rankdata
import scikit_posthocs as sp
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt


In [3]:
df=pd.read_csv('https://raw.githubusercontent.com/muajnstu/DSK-Chain-to-predict-diabeties-/refs/heads/main/FWR-Correction.csv')
display(df.head())

,Classifiers,GB,Unnamed: 2,Unnamed: 3,LGBM,Unnamed: 5,Unnamed: 6,ET,Unnamed: 8,Unnamed: 9,...,Unnamed: 39,PT,Unnamed: 41,Unnamed: 42,PA,Unnamed: 44,Unnamed: 45,SDG,Unnamed: 47,Unnamed: 48
0,Fold,Accuracy,AUC,F-Measure,Accuracy,AUC,F-Measure,Accuracy,AUC,F-Measure,...,F-Measure,Accuracy,AUC,F-Measure,Accuracy,AUC,F-Measure,Accuracy,AUC,F-Measure
1,Fold 1,0.8193,0.7253,0.5148,0.8193,0.6935,0.5383,0.738,0.4648,0.4961,...,0.0902,0.8133,0,0.4485,0.8087,0,0.4622,0.8163,0,0.4869
2,Fold 2,0.8163,0.7286,0.4869,0.8102,0.705,0.4839,0.7425,0.4415,0.4861,...,0.0469,0.494,0,0.3955,0.8133,0,0.4485,0.6657,0,0.4462
3,Fold 3,0.8193,0.6639,0.4953,0.8208,0.6516,0.5279,0.741,0.4446,0.5098,...,0.0611,0.7982,0,0.4781,0.8148,0,0.4569,0.6687,0,0.56
4,Fold 4,0.8223,0.7197,0.5103,0.8163,0.7019,0.519,0.7364,0.4958,0.4649,...,0.0923,0.7892,0,0.537,0.8133,0,0.4564,0.3057,0,0.3044


In [4]:
print(df.columns.tolist())

['Classifiers ', 'GB', 'Unnamed: 2', 'Unnamed: 3', ' LGBM ', 'Unnamed: 5', 'Unnamed: 6', ' ET', 'Unnamed: 8', 'Unnamed: 9', 'Bagging', 'Unnamed: 11', 'Unnamed: 12', 'DT', 'Unnamed: 14', 'Unnamed: 15', 'NB', 'Unnamed: 17', 'Unnamed: 18', 'RF', 'Unnamed: 20', 'Unnamed: 21', 'LDA', 'Unnamed: 23', 'Unnamed: 24', 'QDA', 'Unnamed: 26', 'Unnamed: 27', 'DSK-chain', 'Unnamed: 29', 'Unnamed: 30', 'XGB', 'Unnamed: 32', 'Unnamed: 33', 'Res-MLP', 'Unnamed: 35', 'Unnamed: 36', 'GrowNet', 'Unnamed: 38', 'Unnamed: 39', 'PT', 'Unnamed: 41', 'Unnamed: 42', 'PA', 'Unnamed: 44', 'Unnamed: 45', 'SDG', 'Unnamed: 47', 'Unnamed: 48']


In [5]:
import pandas as pd
import numpy as np
from scipy.stats import friedmanchisquare, rankdata

df.columns = df.columns.str.strip()

accuracy_columns = [
    'GB', 'LGBM', 'ET',
    'Bagging', 'DT',
    'NB', 'RF', 'LDA', 'QDA', 'DSK-chain', 'XGB', 'Res-MLP','GrowNet','PT', 'PA','SDG'
]

# Filter out non-numeric rows
mask = df[accuracy_columns].apply(lambda x: pd.to_numeric(x, errors='coerce')).notnull().all(axis=1)
filtered_df = df[accuracy_columns][mask]

accuracy_data = [filtered_df[col].astype(float).values for col in accuracy_columns]

# Friedman test
statistic, p_value = friedmanchisquare(*accuracy_data)
print("Friedman Chi-square statistic:", statistic)
print("p-value:", p_value)

accuracy_matrix = np.array(accuracy_data).T
ranks = np.array([rankdata(-row) for row in accuracy_matrix])
avg_ranks = ranks.mean(axis=0)

rank_df = pd.DataFrame({
    'Classifier': accuracy_columns,
    'Average Rank': avg_ranks
}).sort_values("Average Rank")

print(rank_df)

Friedman Chi-square statistic: 130.69815498154983
p-value: 1.5520095047358267e-20
   Classifier  Average Rank
9   DSK-chain          1.00
0          GB          3.05
12    GrowNet          3.20
1        LGBM          4.20
11    Res-MLP          5.55
7         LDA          6.85
10        XGB          7.05
14         PA          7.20
13         PT          9.60
6          RF         10.40
15        SDG         10.60
3     Bagging         11.50
2          ET         11.70
4          DT         13.20
8         QDA         15.10
5          NB         15.80


In [6]:
# Friedman test
statistic, p_value = friedmanchisquare(*accuracy_data)

accuracy_matrix = np.array(accuracy_data).T
ranks = np.array([rankdata(-row) for row in accuracy_matrix])
avg_ranks = ranks.mean(axis=0)

best_idx = np.argmin(avg_ranks)
best_model = accuracy_columns[best_idx]

# Nemenyi post hoc p-values
p_values_matrix = sp.posthoc_nemenyi_friedman(accuracy_matrix)
p_values = p_values_matrix.iloc[best_idx].values

# Compute z-values
k = len(accuracy_columns)
N = accuracy_matrix.shape[0]
q = np.sqrt(k*(k+1)/(6*N))
z_values = np.abs(avg_ranks - avg_ranks[best_idx]) / q

# Adjust p-values (Bonferroni, Holm, Hochberg)
p_bonf = multipletests(p_values, method='bonferroni')[1]
p_holm = multipletests(p_values, method='holm')[1]
p_hoch = multipletests(p_values, method='h')[1]

# Holland correction
sorted_idx = np.argsort(p_values)
p_sorted = p_values[sorted_idx]
p_holland_sorted = [1 - (1 - p_sorted[i])**(k - i) for i in range(k)]
p_holland = np.empty_like(p_holland_sorted)
p_holland[sorted_idx] = p_holland_sorted

# Combine results
result_df = pd.DataFrame({
    'Model': accuracy_columns,
    'Ranking': avg_ranks,
    'z': z_values,
    'p': p_values,
    'p_Bonf': p_bonf,
    'p_Holm': p_holm,
    'p_Hochberg': p_hoch,
    'p_Holland': p_holland
}).sort_values("Ranking").reset_index(drop=True)

print("Best Model:", best_model)
display(result_df)


Best Model: DSK-chain


,Model,Ranking,z,p,p_Bonf,p_Holm,p_Hochberg,p_Holland
0,DSK-chain,1.00,0.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
1,GB,3.05,0.962820,9.998806e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
2,GrowNet,3.20,1.033270,9.997139e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
3,LGBM,4.20,1.502938,9.824447e-01,1.000000e+00,1.000000e+00,1.000000e+00,9.999999e-01
4,Res-MLP,5.55,2.136990,7.411234e-01,1.000000e+00,1.000000e+00,1.000000e+00,9.988373e-01
5,LDA,6.85,2.747559,3.000684e-01,1.000000e+00,1.000000e+00,1.000000e+00,8.824200e-01
6,XGB,7.05,2.841493,2.453749e-01,1.000000e+00,1.000000e+00,1.000000e+00,8.606463e-01
7,PA,7.20,2.911943,2.088186e-01,1.000000e+00,1.000000e+00,1.000000e+00,8.464647e-01
8,PT,9.60,4.039147,5.447972e-03,8.716754e-02,4.903174e-02,4.903174e-02,4.797672e-02
9,RF,10.40,4.414881,1.102897e-03,1.764635e-02,1.102897e-02,1.102897e-02,1.097439e-02


In [7]:
from google.colab import files

result_df.to_csv('statistical_results.csv', index=False)
files.download('statistical_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
import altair as alt

chart = alt.Chart(rank_df).mark_line().encode(
    x='Classifier',
    y='Average Rank'
).properties(
    width=800,
    height=500
)

# Display the chart
chart

alt.Chart(...)